In [48]:
#first I'll create a dictionary representing S_box
#the dictionary will be same as given in the research paper
S_val = [bin(0xc,4), bin(0x5,4), bin(0x6,4), bin(0xb,4), bin(0x9,4), bin(0x0,4), bin(0xa,4), bin(0xd,4), bin(0x3,4),
    bin(0xe,4), bin(0xf,4), bin(0x8,4), bin(0x4,4), bin(0x7,4), bin(0x1,4), bin(0x2,4)]
S_box = Dict()
for i in range(0,16)
    S_box[bin(i,4)] = S_val[i+1]
end
println("The S_box: ")
for i in 0:15
    println(hex(i,1),"--->",hex(parse(Int,S_box[bin(i,4)],2),1))
end
println()
#now I'll create the Permutation to represent P_box from the research paper
P_val = [0,16,32,48,1,17,33,49,2,18,34,50,3,19,35,51,4,20,36,52,5,21,37,53,6,22,38,54,7,23,39,55,8,24,40,56,9,25,41,57,
10,26,42,58,11,27,43,59,12,28,44,60,13,29,45,61,14,30,46,62,15,31,47,63]
P_box = Dict()
for i in range(0,64)
    P_box[i] = P_val[i+1]
end
println("The P_box: ")
for i in 0:63
    println(i,"--->",P_box[i])
end

The S_box: 
0--->c
1--->5
2--->6
3--->b
4--->9
5--->0
6--->a
7--->d
8--->3
9--->e
a--->f
b--->8
c--->4
d--->7
e--->1
f--->2

The P_box: 
0--->0
1--->16
2--->32
3--->48
4--->1
5--->17
6--->33
7--->49
8--->2
9--->18
10--->34
11--->50
12--->3
13--->19
14--->35
15--->51
16--->4
17--->20
18--->36
19--->52
20--->5
21--->21
22--->37
23--->53
24--->6
25--->22
26--->38
27--->54
28--->7
29--->23
30--->39
31--->55
32--->8
33--->24
34--->40
35--->56
36--->9
37--->25
38--->41
39--->57
40--->10
41--->26
42--->42
43--->58
44--->11
45--->27
46--->43
47--->59
48--->12
49--->28
50--->44
51--->60
52--->13
53--->29
54--->45
55--->61
56--->14
57--->30
58--->46
59--->62
60--->15
61--->31
62--->47
63--->63


In [57]:
#function definitions to be used in the program 
"""
a very subtle point is the bit positioning starts from 0 form RHS in the paper but in implemetation in julia
the positioning will be done from 1 from LHS, so have to take care of conversion.
"""

#my definition of xor
function xor(binstr1,binstr2)
    l1 = length(binstr1)
    l2 = length(binstr2)
    if l1<l2
        bin(parse(Int128,binstr1,2)$parse(Int128,binstr2,2),l2) #Int128 to avoid overlflow
    else
        bin(parse(Int128,binstr1,2)$parse(Int128,binstr2,2),l1)
    end
end

#definition of the sboxLayer(state)
function sboxLayer(state)
    new_state = ""
    for i in 4:4:64
        new_nibble = S_box[state[i-3:i]]
        new_state = join([new_state,new_nibble]) #join appends the strings inside array
    end
    state = new_state
end

#definition of the pLayer(state)
function pLayer(state)
    l = length(state)
    newStateArray = fill('0',l)
    for i in 1:l
        newStateArray[P_box[l-i]+1] = state[l-i+1]
    end
    state = join(newStateArray)
end

#now the defintion of the genKeys(K_initial)
# K is 80bits long
function keyUpdate(K,i)
    #first rortate by 61 bits to the left
    K_new = rotate_str(K, 61)
    #update the 4 left more bits as given
    K_l4 = S_box[K_new[1:4]]
    K_new = join([K_l4,K_new[5:end]])
    #do xor of K_new[20:16] from right hand side and round counter
    K_in = xor(K_new[61:65],bin(i,5))
    K_new = join([K_new[1:60],K_in,K_new[66:end]])
end

#generic function to rotate the strings
function rotate_str(str,i)
    newstr = ""
    newstr = join([newstr,str[(i+1)%Int(length(str)):end]])
    newstr = join([newstr,str[1:(i%Int(length(str)))]])
end

function genKeys(K_initial)
    k_array = fill("",32)
    k_array[1] = K_initial
    for i in 2:32
        k_array[i] = keyUpdate(k_array[i-1],i-1)
    end
    roundkeys = fill("",32)
    for i in 1:32
        roundkeys[i] = k_array[i][1:64]
    end
    roundkeys
end

genKeys (generic function with 1 method)

In [58]:
rounds = 31
#genrateKeys(K) will be a function which will take in intial key and generate round keys
#the algo is
#genKeys(K_initial)
#for i in 1:31
# xor(state,roundkey_i)  --> this will update the "state"
# sboxLayer(state)       --> this will update state again
# pLaye(state)           --> this will update state again
#end
#xor(state,K_32)

k_initial_hex = 0x0 #key in hex
K_initial = bin(k_initial_hex, 80)
state = bin(0,64)
println("Plaintext: ", state)
keys = genKeys(K_initial)
#putting everything together
for i in 1:rounds
    println("round ",i)
    println("round key: ", hex(parse(Int128,keys[i],2),16))
    state = xor(state,keys[i])
    println("after xor with key: ", hex(parse(Int128,state,2),16))
    state = sboxLayer(state)
    println("after sBoxLaye: ",hex(parse(Int128,state,2),16))
    state = pLayer(state)
    println("round output: ", hex(parse(Int128,state,2),16))
    println()
    println()
end
state = xor(state,keys[32])
println("Ciphertext: ", state)

Plaintext: 0000000000000000000000000000000000000000000000000000000000000000
round 1
round key: 0000000000000000
after xor with key: 0000000000000000
after sBoxLaye: cccccccccccccccc
round output: ffffffff00000000


round 2
round key: c000000000000000
after xor with key: 3fffffff00000000
after sBoxLaye: b2222222cccccccc
round output: 80ff00ffff008000


round 3
round key: 5000180000000001
after xor with key: d0ff18ffff008001
after sBoxLaye: 7c22532222cc3cc5
round output: 4036c837b7c88c09


round 4
round key: 60000a0003000001
after xor with key: 2036c237b4c88c08
after sBoxLaye: 6cba46bd894334c3
round output: 73c2cd26b6192359


round 5
round key: b0000c0001400062
after xor with key: c3c2c126b759233b
after sBoxLaye: 4b46456a8d0e6bb8
round output: 41d7be58531e4446


round 6
round key: 900016000180002a
after xor with key: d1d7a858529e446c
after sBoxLaye: 757df30306e199a4
round output: 182ef861ad62fd1c


round 7
round key: 0001920002c00033
after xor with key: 182f6a61afa2fd2f
after sBoxLaye: 5

In [39]:
parse(Int128,"0100000011101101110101011110110111010110011110101111110001001010",2)

4678630805390163018

In [30]:
parse(Int128,"5579c1387b228445",16)

6159166413734052933

In [31]:
hex(1234354464677543,1)

"462a38b64dea7"